In [1]:
# !pip install qdrant-client openai

In [2]:
import os
from dotenv import load_dotenv

from qdrant_client import QdrantClient
from qdrant_client.http import models

load_dotenv('./.env')

True

In [3]:
# create your client
client = QdrantClient(
    os.getenv("QDRANT_HOST"),
    api_key=os.getenv("QDRANT_API_KEY")
)

In [4]:
# create your collection
collection_config = models.VectorParams(
    size=1536,
    distance=models.Distance.COSINE
)

client.create_collection(
    collection_name=os.getenv('QDRANT_COLLECTION_NAME'),
    vectors_config=collection_config
)

True

In [5]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='my_collection')])

### Vector Store

In [6]:
from langchain.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

In [7]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant(
    client=client,
    collection_name=os.getenv('QDRANT_COLLECTION_NAME'),
    embeddings=embeddings
)

In [8]:
def get_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

with open("story.txt") as f:
    raw_text = f.read()

texts = get_chunks(raw_text)

vectorstore.add_texts(texts)

['58bb1292a1844ed58b4b3269698aadb3',
 'a12ec2b635ac46a8918672c69d253689',
 'ec8c9d021ec34c7dbf60b1b7931bbe61',
 '77513eeac24941c097da9c5074573cb7',
 '9fcd0abb20bc46c68f8030a5e1bcb01e']

In [9]:
#client.delete_collection(collection_name="my_collection")